# Option D: Custom Training Pipeline

**Module:** 4.3 - Capstone Project (Domain 4: Production AI)
**Time:** 35-45 hours total
**Difficulty:** ⭐⭐⭐⭐⭐

---

## 🎯 Project Overview

Build infrastructure for continuous model improvement:
- **Data Pipeline** - Collection, curation, preprocessing
- **Training System** - SFT, DPO, RLHF support
- **Evaluation Framework** - Automated benchmarking
- **Model Registry** - Versioning and comparison
- **Deployment Automation** - A/B testing ready

---

## 🎯 Learning Objectives

By completing this project, you will:
- [ ] Build a complete data curation pipeline
- [ ] Implement multiple training approaches (SFT, DPO)
- [ ] Create automated evaluation systems
- [ ] Build model versioning and registry
- [ ] Implement deployment with A/B testing

---

## 🌍 Real-World Context

Training pipelines are the backbone of AI companies:

| Company | Pipeline Focus | Scale |
|---------|---------------|-------|
| **OpenAI** | RLHF, Constitutional AI | Millions of samples |
| **Anthropic** | CAI, Preference Learning | Safety-focused |
| **Meta** | SFT + DPO | Open-source models |
| **Google** | Multi-task, Instruction tuning | Massive scale |

Building your own pipeline gives you control over:
- Data quality and bias
- Training efficiency
- Model behavior
- Continuous improvement

---

## 🧒 ELI5: What is a Training Pipeline?

> **Imagine you're training a new employee.** You don't just throw them into work:
>
> 1. **Gather training materials** (data collection)
> 2. **Organize the content** (data curation)
> 3. **Conduct training sessions** (model training)
> 4. **Give quizzes to check progress** (evaluation)
> 5. **Promote good performers** (model selection)
> 6. **Deploy to their role** (serving)
> 7. **Get feedback and retrain** (continuous improvement)
>
> **A training pipeline automates this entire process** for AI models. Feed in data, get out improved models - automatically, reproducibly, at scale.

---

## 🏗️ System Architecture

```
┌─────────────────────────────────────────────────────────────────────┐
│                    Custom Training Pipeline                          │
├─────────────────────────────────────────────────────────────────────┤
│                                                                      │
│  ┌────────────┐   ┌────────────┐   ┌────────────┐   ┌────────────┐ │
│  │   Data     │──▶│   Data     │──▶│  Training  │──▶│   Model    │ │
│  │ Collection │   │  Curation  │   │   Loop     │   │  Registry  │ │
│  └────────────┘   └────────────┘   └──────┬─────┘   └──────┬─────┘ │
│                                          │                │        │
│                                          ▼                ▼        │
│  ┌────────────────────────────┐    ┌────────────┐   ┌──────────┐  │
│  │      Evaluation Suite      │◀───│   Model    │──▶│  Deploy  │  │
│  │ • Benchmarks               │    │ Comparison │   │  A/B     │  │
│  │ • Safety Checks            │    └────────────┘   │  Testing │  │
│  │ • Quality Metrics          │                     └──────────┘  │
│  └────────────────────────────┘                                   │
│                                                                      │
│  ┌────────────────────────────────────────────────────────────┐    │
│  │                    Experiment Tracking                      │    │
│  │              (MLflow / Weights & Biases)                   │    │
│  └────────────────────────────────────────────────────────────┘    │
│                                                                      │
└─────────────────────────────────────────────────────────────────────┘
```

---

## Part 1: Data Pipeline

In [ ]:
# Data Pipeline Foundation

import torch
from datetime import datetime
from typing import List, Dict, Any, Optional, Iterator
from dataclasses import dataclass, field
from pathlib import Path
import json
import hashlib

print("🚀 OPTION D: CUSTOM TRAINING PIPELINE")
print("="*60)
print(f"Date: {datetime.now().strftime('%Y-%m-%d %H:%M')}")
print(f"\nGPU: {torch.cuda.get_device_name(0) if torch.cuda.is_available() else 'Not available'}")

@dataclass
class TrainingSample:
    """A single training sample."""
    id: str
    instruction: str
    input: str = ""
    output: str = ""
    system_prompt: str = ""
    metadata: Dict[str, Any] = field(default_factory=dict)
    
    def to_dict(self) -> Dict:
        return {
            "id": self.id,
            "instruction": self.instruction,
            "input": self.input,
            "output": self.output,
            "system_prompt": self.system_prompt,
            "metadata": self.metadata,
        }
    
    @classmethod
    def from_dict(cls, data: Dict) -> "TrainingSample":
        return cls(
            id=data.get("id", hashlib.md5(data["instruction"].encode()).hexdigest()[:8]),
            instruction=data["instruction"],
            input=data.get("input", ""),
            output=data.get("output", ""),
            system_prompt=data.get("system_prompt", ""),
            metadata=data.get("metadata", {}),
        )

@dataclass
class PreferencePair:
    """A preference pair for DPO training."""
    id: str
    instruction: str
    input: str = ""
    chosen: str = ""  # Preferred response
    rejected: str = ""  # Non-preferred response
    metadata: Dict[str, Any] = field(default_factory=dict)

class DataCollector:
    """
    Collects training data from various sources.
    """
    
    def __init__(self, output_dir: str = "./data/collected"):
        self.output_dir = Path(output_dir)
        self.output_dir.mkdir(parents=True, exist_ok=True)
        self.samples: List[TrainingSample] = []
    
    def add_sample(self, sample: TrainingSample):
        """Add a single sample."""
        self.samples.append(sample)
    
    def load_from_jsonl(self, path: str) -> int:
        """Load samples from JSONL file."""
        count = 0
        with open(path, 'r') as f:
            for line in f:
                data = json.loads(line)
                self.samples.append(TrainingSample.from_dict(data))
                count += 1
        print(f"✅ Loaded {count} samples from {path}")
        return count
    
    def load_from_hf_dataset(self, dataset_name: str, split: str = "train") -> int:
        """Load from Hugging Face datasets."""
        from datasets import load_dataset
        
        dataset = load_dataset(dataset_name, split=split)
        count = 0
        
        for item in dataset:
            # Handle various dataset formats
            sample = TrainingSample(
                id=item.get("id", str(count)),
                instruction=item.get("instruction", item.get("question", "")),
                input=item.get("input", item.get("context", "")),
                output=item.get("output", item.get("answer", "")),
                metadata={"source": dataset_name}
            )
            self.samples.append(sample)
            count += 1
        
        print(f"✅ Loaded {count} samples from {dataset_name}")
        return count
    
    def save(self, filename: str = "collected.jsonl"):
        """Save all collected samples."""
        output_path = self.output_dir / filename
        with open(output_path, 'w') as f:
            for sample in self.samples:
                f.write(json.dumps(sample.to_dict()) + "\n")
        print(f"✅ Saved {len(self.samples)} samples to {output_path}")

print("\n✅ Data collection system defined")

In [ ]:
# Data Curation System

class DataCurator:
    """
    Curates and filters training data for quality.
    """
    
    def __init__(self):
        self.filters = []
        self.transformations = []
    
    def add_filter(self, filter_fn, name: str = ""):
        """Add a filter function. Returns True to keep, False to discard."""
        self.filters.append((filter_fn, name))
    
    def add_transformation(self, transform_fn, name: str = ""):
        """Add a transformation function."""
        self.transformations.append((transform_fn, name))
    
    def curate(self, samples: List[TrainingSample]) -> List[TrainingSample]:
        """Apply all filters and transformations."""
        curated = []
        filter_stats = {name: 0 for _, name in self.filters}
        
        for sample in samples:
            # Apply filters
            keep = True
            for filter_fn, name in self.filters:
                if not filter_fn(sample):
                    filter_stats[name] += 1
                    keep = False
                    break
            
            if not keep:
                continue
            
            # Apply transformations
            for transform_fn, _ in self.transformations:
                sample = transform_fn(sample)
            
            curated.append(sample)
        
        # Print stats
        print(f"\n📊 Curation Stats:")
        print(f"   Input: {len(samples)}")
        print(f"   Output: {len(curated)}")
        for name, count in filter_stats.items():
            if count > 0:
                print(f"   Filtered by {name}: {count}")
        
        return curated

# Common filters
def min_length_filter(min_len: int = 10):
    """Filter samples with output shorter than min_len."""
    def filter_fn(sample: TrainingSample) -> bool:
        return len(sample.output) >= min_len
    return filter_fn

def max_length_filter(max_len: int = 4096):
    """Filter samples with total length exceeding max_len."""
    def filter_fn(sample: TrainingSample) -> bool:
        total = len(sample.instruction) + len(sample.input) + len(sample.output)
        return total <= max_len
    return filter_fn

def no_pii_filter():
    """Filter samples containing potential PII."""
    import re
    pii_patterns = [
        r'\b[A-Za-z0-9._%+-]+@[A-Za-z0-9.-]+\.[A-Z|a-z]{2,}\b',  # Email
        r'\b\d{3}[-.]?\d{3}[-.]?\d{4}\b',  # Phone
        r'\b\d{3}-\d{2}-\d{4}\b',  # SSN
    ]
    
    def filter_fn(sample: TrainingSample) -> bool:
        text = f"{sample.instruction} {sample.input} {sample.output}"
        for pattern in pii_patterns:
            if re.search(pattern, text):
                return False
        return True
    return filter_fn

def deduplication_filter(seen_hashes: set = None):
    """Filter duplicate samples."""
    if seen_hashes is None:
        seen_hashes = set()
    
    def filter_fn(sample: TrainingSample) -> bool:
        content = f"{sample.instruction}{sample.output}"
        content_hash = hashlib.md5(content.encode()).hexdigest()
        if content_hash in seen_hashes:
            return False
        seen_hashes.add(content_hash)
        return True
    return filter_fn

print("✅ Data curation system defined")

---

## Part 2: Training System

In [ ]:
# Training Configuration

@dataclass
class TrainingConfig:
    """Configuration for training runs."""
    # Model
    base_model: str = "meta-llama/Llama-3.3-8B-Instruct"
    
    # LoRA
    lora_r: int = 64
    lora_alpha: int = 128
    lora_dropout: float = 0.05
    target_modules: List[str] = field(default_factory=lambda: [
        "q_proj", "k_proj", "v_proj", "o_proj",
        "gate_proj", "up_proj", "down_proj"
    ])
    
    # Training
    training_method: str = "sft"  # sft, dpo
    num_epochs: int = 3
    batch_size: int = 4
    gradient_accumulation: int = 4
    learning_rate: float = 2e-4
    max_seq_length: int = 2048
    warmup_ratio: float = 0.03
    
    # DPO specific
    dpo_beta: float = 0.1
    
    # Output
    output_dir: str = "./models/trained"
    run_name: str = ""
    
    def to_dict(self) -> Dict:
        return {
            "base_model": self.base_model,
            "lora_r": self.lora_r,
            "lora_alpha": self.lora_alpha,
            "training_method": self.training_method,
            "num_epochs": self.num_epochs,
            "batch_size": self.batch_size,
            "learning_rate": self.learning_rate,
            "max_seq_length": self.max_seq_length,
        }

class TrainingPipeline:
    """
    Unified training pipeline supporting SFT and DPO.
    """
    
    def __init__(self, config: TrainingConfig):
        self.config = config
        self.model = None
        self.tokenizer = None
        self.run_id = f"{config.run_name or config.training_method}_{datetime.now().strftime('%Y%m%d_%H%M%S')}"
    
    def prepare_model(self):
        """Load and prepare model for training."""
        from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig
        from peft import LoraConfig, get_peft_model, prepare_model_for_kbit_training
        
        print(f"\n🔄 Loading model: {self.config.base_model}")
        
        # Quantization
        bnb_config = BitsAndBytesConfig(
            load_in_4bit=True,
            bnb_4bit_compute_dtype=torch.bfloat16,
            bnb_4bit_quant_type="nf4",
            bnb_4bit_use_double_quant=True,
        )
        
        # Load model
        self.model = AutoModelForCausalLM.from_pretrained(
            self.config.base_model,
            quantization_config=bnb_config,
            device_map="auto",
            trust_remote_code=True,
        )
        
        self.tokenizer = AutoTokenizer.from_pretrained(self.config.base_model)
        self.tokenizer.pad_token = self.tokenizer.eos_token
        
        # Prepare for k-bit training
        self.model = prepare_model_for_kbit_training(self.model)
        
        # Apply LoRA
        lora_config = LoraConfig(
            r=self.config.lora_r,
            lora_alpha=self.config.lora_alpha,
            lora_dropout=self.config.lora_dropout,
            target_modules=self.config.target_modules,
            bias="none",
            task_type="CAUSAL_LM",
        )
        
        self.model = get_peft_model(self.model, lora_config)
        self.model.print_trainable_parameters()
        
        print("✅ Model ready for training")
    
    def train_sft(self, train_data: List[TrainingSample]):
        """Supervised Fine-Tuning."""
        from transformers import TrainingArguments
        from trl import SFTTrainer
        from datasets import Dataset
        
        print(f"\n🏋️ Starting SFT training with {len(train_data)} samples")
        
        if self.model is None:
            self.prepare_model()
        
        # Prepare dataset
        def format_sample(sample: TrainingSample) -> str:
            messages = [
                {"role": "system", "content": sample.system_prompt or "You are a helpful assistant."},
                {"role": "user", "content": sample.instruction + (f"\n\n{sample.input}" if sample.input else "")},
                {"role": "assistant", "content": sample.output}
            ]
            return self.tokenizer.apply_chat_template(messages, tokenize=False)
        
        formatted = [format_sample(s) for s in train_data]
        dataset = Dataset.from_dict({"text": formatted})
        
        # Training arguments
        output_dir = f"{self.config.output_dir}/{self.run_id}"
        
        training_args = TrainingArguments(
            output_dir=output_dir,
            num_train_epochs=self.config.num_epochs,
            per_device_train_batch_size=self.config.batch_size,
            gradient_accumulation_steps=self.config.gradient_accumulation,
            learning_rate=self.config.learning_rate,
            lr_scheduler_type="cosine",
            warmup_ratio=self.config.warmup_ratio,
            logging_steps=10,
            save_steps=100,
            bf16=True,
            gradient_checkpointing=True,
            report_to=[],  # Disable wandb by default
        )
        
        # Trainer
        trainer = SFTTrainer(
            model=self.model,
            args=training_args,
            train_dataset=dataset,
            tokenizer=self.tokenizer,
            dataset_text_field="text",
            max_seq_length=self.config.max_seq_length,
        )
        
        # Train
        trainer.train()
        trainer.save_model(output_dir)
        
        print(f"\n✅ Training complete. Model saved to {output_dir}")
        return output_dir
    
    def train_dpo(self, preference_data: List[PreferencePair]):
        """Direct Preference Optimization training."""
        from transformers import TrainingArguments
        from trl import DPOTrainer
        from datasets import Dataset
        
        print(f"\n🏋️ Starting DPO training with {len(preference_data)} pairs")
        
        if self.model is None:
            self.prepare_model()
        
        # Prepare dataset
        dataset_dict = {
            "prompt": [p.instruction + (f"\n\n{p.input}" if p.input else "") for p in preference_data],
            "chosen": [p.chosen for p in preference_data],
            "rejected": [p.rejected for p in preference_data],
        }
        dataset = Dataset.from_dict(dataset_dict)
        
        output_dir = f"{self.config.output_dir}/{self.run_id}"
        
        training_args = TrainingArguments(
            output_dir=output_dir,
            num_train_epochs=self.config.num_epochs,
            per_device_train_batch_size=self.config.batch_size // 2,  # DPO needs more memory
            gradient_accumulation_steps=self.config.gradient_accumulation * 2,
            learning_rate=self.config.learning_rate / 10,  # Lower LR for DPO
            bf16=True,
            gradient_checkpointing=True,
            report_to=[],
        )
        
        # DPO Trainer
        trainer = DPOTrainer(
            model=self.model,
            args=training_args,
            beta=self.config.dpo_beta,
            train_dataset=dataset,
            tokenizer=self.tokenizer,
            max_length=self.config.max_seq_length,
        )
        
        trainer.train()
        trainer.save_model(output_dir)
        
        print(f"\n✅ DPO training complete. Model saved to {output_dir}")
        return output_dir

print("✅ Training pipeline defined")

---

## Part 3: Evaluation & Model Registry

In [ ]:
# Model Registry

@dataclass
class ModelVersion:
    """A versioned model in the registry."""
    id: str
    name: str
    version: str
    path: str
    base_model: str
    training_config: Dict
    metrics: Dict[str, float]
    created_at: datetime
    status: str = "active"  # active, deprecated, archived
    notes: str = ""

class ModelRegistry:
    """
    Registry for tracking model versions.
    """
    
    def __init__(self, storage_path: str = "./registry"):
        self.storage_path = Path(storage_path)
        self.storage_path.mkdir(parents=True, exist_ok=True)
        self.models: Dict[str, List[ModelVersion]] = {}
        self._load()
    
    def _load(self):
        """Load registry from disk."""
        registry_file = self.storage_path / "registry.json"
        if registry_file.exists():
            with open(registry_file) as f:
                data = json.load(f)
            for name, versions in data.items():
                self.models[name] = [
                    ModelVersion(
                        **{**v, "created_at": datetime.fromisoformat(v["created_at"])}
                    ) for v in versions
                ]
    
    def _save(self):
        """Save registry to disk."""
        registry_file = self.storage_path / "registry.json"
        data = {}
        for name, versions in self.models.items():
            data[name] = [
                {**v.__dict__, "created_at": v.created_at.isoformat()}
                for v in versions
            ]
        with open(registry_file, "w") as f:
            json.dump(data, f, indent=2)
    
    def register(
        self,
        name: str,
        path: str,
        base_model: str,
        training_config: Dict,
        metrics: Dict[str, float],
        notes: str = ""
    ) -> ModelVersion:
        """Register a new model version."""
        if name not in self.models:
            self.models[name] = []
        
        version = f"v{len(self.models[name]) + 1}"
        
        model_version = ModelVersion(
            id=f"{name}_{version}",
            name=name,
            version=version,
            path=path,
            base_model=base_model,
            training_config=training_config,
            metrics=metrics,
            created_at=datetime.now(),
            notes=notes,
        )
        
        self.models[name].append(model_version)
        self._save()
        
        print(f"✅ Registered {name} {version}")
        return model_version
    
    def get_latest(self, name: str) -> Optional[ModelVersion]:
        """Get the latest version of a model."""
        if name not in self.models or not self.models[name]:
            return None
        active = [m for m in self.models[name] if m.status == "active"]
        return active[-1] if active else None
    
    def get_best(self, name: str, metric: str) -> Optional[ModelVersion]:
        """Get the best version by a specific metric."""
        if name not in self.models:
            return None
        
        versions = [m for m in self.models[name] if metric in m.metrics and m.status == "active"]
        if not versions:
            return None
        
        return max(versions, key=lambda m: m.metrics[metric])
    
    def compare(self, name: str) -> str:
        """Compare all versions of a model."""
        if name not in self.models:
            return f"No model named {name}"
        
        lines = [f"\n📊 Model Comparison: {name}", "="*60]
        
        for v in self.models[name]:
            lines.append(f"\n{v.version} ({v.status})")
            lines.append(f"  Created: {v.created_at.strftime('%Y-%m-%d %H:%M')}")
            lines.append(f"  Metrics:")
            for metric, value in v.metrics.items():
                lines.append(f"    {metric}: {value:.4f}")
        
        return "\n".join(lines)

print("✅ Model registry defined")

---

## 🎉 Checkpoint & Next Steps

You now have the foundation for a Custom Training Pipeline:

- ✅ Data collection and curation
- ✅ SFT and DPO training support
- ✅ Model registry with versioning
- ✅ Evaluation framework (basic)

### Implementation Roadmap

**Week 1-2:** Data pipeline
- [ ] Add more data sources
- [ ] Build quality scoring
- [ ] Implement synthetic data generation

**Week 3:** Training system
- [ ] Add distributed training support
- [ ] Implement curriculum learning
- [ ] Add RLHF support

**Week 4:** Evaluation
- [ ] Create comprehensive benchmarks
- [ ] Add safety evaluation
- [ ] Implement A/B testing

**Week 5-6:** Automation & docs
- [ ] Build CI/CD pipeline
- [ ] Add monitoring
- [ ] Document everything

---

In [ ]:
# Cleanup
import gc

if torch.cuda.is_available():
    torch.cuda.empty_cache()
gc.collect()

print("✅ Ready to build your Training Pipeline!")